In [1]:
import os
import json
import pandas as pd
from pathlib import Path

pd.options.display.float_format = "{:,.4f}".format
base_dir = Path(".")  # or wherever your top-level folders live
datasets = {
    "joined": "Joined",
    "private_data": "Private",
}

rows = []
for ds_key, ds_name in datasets.items():
    ds_path = base_dir / ds_key
    if not ds_path.is_dir():
        print(f"[!] dataset folder not found: {ds_path}")
        continue

    # each model is just a subfolder
    for model_folder in sorted(ds_path.iterdir()):
        if not model_folder.is_dir():
            continue
        model_name = model_folder.name
        print(model_name)
        # each compression method is a sub-subfolder
        
        # attempt to load best_metrics.json
        metrics_path = model_folder / "best_model_metrics.json"
        if not metrics_path.is_file():
            # skip if no metrics
            continue

        with open(metrics_path, "r") as f:
            metrics = json.load(f)
    

        # pick out the stats you care about
        rows.append(
            {
                "dataset": ds_name,
                "model": model_name,
                "1_f1": metrics.get("1", {}).get("f1-score"),
                "1_recall": metrics.get("1", {}).get("recall"),
                "1_precision": metrics.get("1", {}).get("precision"),
                "macro_f1": metrics.get("macro avg", {}).get("f1-score"),
                "roc_auc": metrics.get("roc_auc_score"),
            }
        )

# build DataFrame
df = pd.DataFrame(rows)

# format numbers
numeric_cols = df.select_dtypes(include=["number"]).columns
for col in numeric_cols:
    if col in ("throughput_model", "throughput_total"):
        # integer throughputs
        df[col] = df[col].round(0).astype("Int64").map(lambda x: f"{x:,}" if pd.notnull(x) else "")
    else:
        # float stats
        df[col] = df[col].map(lambda x: f"{x:,.4f}" if pd.notnull(x) else "")

# display
print(df.to_string(index=False))

bert_mini
bert_mini_domain_masked
bert_small_baseline
bert_tiny_ablation_best_group
bert_mini
bert_mini_domain_masked
bert_small_baseline
bert_tiny
dataset                         model   1_f1 1_recall 1_precision macro_f1 roc_auc
 Joined                     bert_mini 0.9327   0.9132      0.9530   0.9567  0.9872
 Joined       bert_mini_domain_masked 0.9293   0.9210      0.9378   0.9544  0.9875
 Joined           bert_small_baseline 0.9322   0.9149      0.9501   0.9563  0.9845
 Joined bert_tiny_ablation_best_group 0.9220   0.8941      0.9516   0.9499  0.9812
Private                     bert_mini 0.8248   0.8312      0.8185   0.9035  0.9632
Private       bert_mini_domain_masked 0.8625   0.8368      0.8899   0.9246  0.9589
Private           bert_small_baseline 0.8648   0.8103      0.9271   0.9261  0.9599
Private                     bert_tiny 0.7747   0.7290      0.8265   0.8767  0.9383
